## IS 455 Fall Semester - Final Project Part 2 - Ron Payne

## Assignment
Using your dataset from Part 1:

Submit code and a writeup (either separate or within the notebook):

Code (70 pts):
* Jupyter notebook with an interactive dashboard that helps an expert explore your dataset thoroughly.
* There should be a "dashboard" type aspect to this - i.e. a linked view exploring your dataset in an interactive way
* Do not delete any cells, just comment them out. Show your work.

Prose (30 pts):
* One paragraph explaining how to use the dashboard you created, to help someone who is not an expert understand your dataset.
* A list of 1 or more contextual datasets you have identified, links to where they reside, and a sentence about why they might be useful in telling the final story. 
    
---

## My Content
XSEDE is an NSF funded project that provides free access to selected academic advanced computing resources for any academic researcher in the U.S. Every quarter, allocation requests are collected, peer-reviewed and awarded. Each allocation award is categorized based on the NSF Field of Science Directorates, including sub-categorations of Divisions within each directorate.   

For my dashboard, I would like to provide the following a heatmap of the XSEDE the quantity of allocations by field of science directorate by year. Upon selecting a heatmap cell, a bar chart under the heatmap updates showing, for the selected year, the quantity of allocations for the divisions of the directorate selected.

For example, if the user selects the heatmap that intersects at the year of 2010 and the field of science directorate named Geosciences, the cell will highlight and the bar chart under the heatmap will dynamically updated to show the number of allocations awarded to research projects in the geosciences directorate grouped by the divisions within Geosciences, which are: Atmosperic Sciences, Earth Sciences, and Ocean Sciences.

There are many XSEDE links that would help with the final story. 
* information about what XSEDE is can be found on the [XSEDE webpage](https://www.xsede.org/)
* information about getting an allocation can be found on the [XSEDE getting started user webpage](https://www.xsede.org/for-users/getting-started)
* information about XSEDE resources (including statistics of usage) is located on the [XSEDE User Portal](https://portal.xsede.org/)

## Coding the Dashbard

Loading libraries and data:

In [2]:
%matplotlib inline
import cartopy
import pandas as pd
import matplotlib.pyplot as plt
import ipyleaflet
import numpy as np
import bqplot
import ipywidgets
import contextily as ctx

In [3]:
awards = pd.read_csv('https://github.com/rpayne615/rpayne615.github.io/raw/master/XSEDE_allocation_award_history_2004-2019.zip')

Checking the data: 

In [4]:
awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19607 entries, 0 to 19606
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Proposal_Number               19469 non-null  object
 1   Grant_Number                  19607 non-null  object
 2   PI_id                         19607 non-null  object
 3   PI_Last_Name                  19607 non-null  object
 4   PI_First_Name                 19607 non-null  object
 5   Organization                  19607 non-null  object
 6   Field_of_Science              19607 non-null  object
 7   Field_of_Science_Division     19607 non-null  object
 8   Field_of_Science_Directorate  19607 non-null  object
 9   Project_Type                  15950 non-null  object
 10  Transaction_Type              19607 non-null  object
 11  Project_Title                 19607 non-null  object
 12  Start_Date                    19607 non-null  object
 13  End_Date        

In [5]:
usage.info()

NameError: name 'usage' is not defined

In [ ]:
awards

Adding a column to the 'awards' dataframe for the year the award started:

In [ ]:
awards['Year'] = pd.DatetimeIndex(awards['Start_Date']).year

Making sure I can get the ipywidget working:

In [ ]:
@ipywidgets.interact(filter=['Field_of_Science_Directorate', 'Project_Type', 'Year'])
def chart(filter):
    awards.pivot_table(values='Grant_Number', index=filter, columns='Transaction_Type', aggfunc='count').plot.barh(stacked=True)
    

coming back to add create the function for updating the widget label based on the heatmap cell selection:

In [ ]:
selectedLabel = ipywidgets.Label(layout=ipywidgets.Layout(display="flex",justify_content="center", width="100%"))
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i,j = change['owner'].selected[0]
        v = heatmap_data.iloc[i,j]
        selectedLabel.value = str(v) + ' allocations for ' + str(fos[i]) + ' in ' + str(year[j])

Making the heatmap for the main component of the dashboard along with the widget label.

In [ ]:
# 1. Data
heatmap_data = awards.pivot_table(values='Grant_Number', 
                                  index='Field_of_Science_Directorate', 
                                  columns='Year', 
                                  aggfunc='count')

fos = heatmap_data.index
year = heatmap_data.columns

# 2. Scales
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Years')
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical', label='FOS')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = heatmap_data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              row = heatmap_data.index,
                              column = heatmap_data.columns,
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# 5. Interactions (some in Marks as well)
heat_map.observe(on_selected, 'selected')


# Put it all together
hm_fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax], 
                       fig_margin={'top':30, 'bottom':30, 'left':300, 'right':60},
                       title="Heatmap of XSEDE Allocations from 2003 to 2019")
xsedeDashboard = ipywidgets.VBox([selectedLabel, hm_fig])
xsedeDashboard

In [ ]:
heatmap_data.columns

In [ ]:
heatmap_data.index

Now that the heatmap is working, I need to start on the secondary dependent plot. I am starting out with a horizontal bar chart of the secondary field of science (division) and the year. This is represents the unique variables in the heatmap cells.

In [ ]:
bc_data = awards.loc[(awards['Year'] == 2016) & (awards['Field_of_Science_Directorate'] == 'Mathematical and Physical Sciences')]

In [ ]:
bc_data.shape

In [ ]:
bc_data

Now for a quick test plot:

In [ ]:
bc_data.pivot_table(values='Grant_Number', index='Field_of_Science_Division', columns='Transaction_Type', aggfunc='count').plot.barh(stacked=True)

Looks good but I realized that I should group the date to make the bqplot or matplotlib plot.

In [ ]:
bc_data_grp = bc_data.groupby('Field_of_Science_Division')['Grant_Number'].count()

In [ ]:
bc_data_grp

In [ ]:
bc_data_grp.index

Let's try a bqplot bar chart.

In [ ]:
# 2 Scales:
x_scb = bqplot.LinearScale()
y_scb = bqplot.OrdinalScale() 

# 3 Axis: 
x_axb = bqplot.Axis(scale = x_scb, label = 'Allocations')
y_axb = bqplot.Axis(scale = y_scb, 
                    orientation = 'vertical', 
                    label='FOS Division')

# 4 Marks:
bc = bqplot.Bars(x=bc_data_grp, y=bc_data_grp.index,
             scales={'x': x_scb, 'y': y_scb}, orientation = 'horizontal')

# 5 No Linking 

# Plot Barchart:
fig_bar = bqplot.Figure(marks = [bc], axes = [x_axb, y_axb])
fig_bar

Well, I can't get this to work so let's try a simple Matplotlib plot.

In [ ]:
plt.barh(bc_data_grp.index, bc_data_grp)
plt.xlabel("Allocations")
plt.ylabel('Field of Science Division')
plt.title('Allocations for FOS_div in year')

Much better. Now I need to make a funtion that can plot based on the field of science directorate and year that is passed into it.

In [ ]:
def bar_chart(fos_dir, year):
    bc_data = awards.loc[(awards['Year'] == year) & (awards['Field_of_Science_Directorate'] == fos_dir)]
    bc_data_grp = bc_data.groupby('Field_of_Science_Division')['Grant_Number'].count()
    
    plt.barh(bc_data_grp.index, bc_data_grp)
    plt.xlabel("Allocations")
    plt.ylabel('Field of Science Division')
    plt.title('Allocations for ' + fos_dir + ' in ' + str(year))


In [ ]:
yr = 2016
dir = 'Engineering'
bar_chart(dir, yr)

Cool, this seems to work. Let's see if I can update the 'on_selected' function to draw the bar chart.

In [ ]:
selectedLabel = ipywidgets.Label(layout=ipywidgets.Layout(display="flex",justify_content="center", width="100%"))
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i,j = change['owner'].selected[0]
        v = heatmap_data.iloc[i,j]
        selectedLabel.value = str(v) + ' allocations for ' + str(fos[i]) + ' in ' + str(year[j])
    bar_chart(fos[i], year[j])

In [ ]:
# 1. Data
heatmap_data = awards.pivot_table(values='Grant_Number', 
                                  index='Field_of_Science_Directorate', 
                                  columns='Year', 
                                  aggfunc='count')

fos = heatmap_data.index
year = heatmap_data.columns

# 2. Scales
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Years')
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical', label='FOS')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = heatmap_data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              row = heatmap_data.index,
                              column = heatmap_data.columns,
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# 5. Interactions (plus what is in Marks)
heat_map.observe(on_selected, 'selected')

# Put it all together
hm_fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax], 
                       fig_margin={'top':30, 'bottom':30, 'left':300, 'right':60},
                       title="Heatmap of XSEDE Allocations from 2003 to 2019")

xsedeDashboard = ipywidgets.VBox([selectedLabel, hm_fig])
xsedeDashboard

Finally got that to work, but it doesn't update or write over the previous bar chart, it just plots a new one below it.

With help (thanks Prof. Naiman!!), the bqplot bar chart is now working, just not with the horizontal orientation. Looks like I found a bug in bqplot! 

So, I can now put it all together with the auto-update of the bar chart with the selction of a heatmap cell. 

In [ ]:
selectedLabel = ipywidgets.Label(layout=ipywidgets.Layout(display="flex",justify_content="center", width="100%"))
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i,j = change['owner'].selected[0]
        v = heatmap_data.iloc[i,j]
        selectedLabel.value = str(v) + ' allocations for ' + str(fos[i]) + ' in ' + str(year[j])
    
    bc_data = awards.loc[(awards['Year'] == year[j]) & (awards['Field_of_Science_Directorate'] == fos[i])]
    bc_data_grp = bc_data.groupby('Field_of_Science_Division')['Grant_Number'].count()
    bc.x = np.array(bc_data_grp.index)
    bc.y = bc_data_grp.values

In [ ]:
# 1. Data
heatmap_data = awards.pivot_table(values='Grant_Number', 
                                  index='Field_of_Science_Directorate', 
                                  columns='Year', 
                                  aggfunc='count')

fos = heatmap_data.index
year = heatmap_data.columns

# 2. Scales
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Years')
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical', label='FOS')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = heatmap_data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              row = heatmap_data.index,
                              column = heatmap_data.columns,
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# 5. Interactions (plus what is in Marks)
heat_map.observe(on_selected, 'selected')

# Put it all together
hm_fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax], 
                       fig_margin={'top':30, 'bottom':30, 'left':300, 'right':60},
                       title="Heatmap of XSEDE Allocations from 2003 to 2019")

In [ ]:
# Bar Chart Creation

# 1. Data
bc_data = awards.loc[(awards['Year'] == 2016) & (awards['Field_of_Science_Directorate'] == 'Social, Behavioral, and Economic Sciences')]
bc_data_grp = bc_data.groupby('Field_of_Science_Division')['Grant_Number'].count()

# 2 Scales:
x_scb = bqplot.OrdinalScale() 
y_scb = bqplot.LinearScale()

# 3 Axis: 
x_axb = bqplot.Axis(scale = x_scb, label = 'FOS Division')
y_axb = bqplot.Axis(scale = y_scb, orientation = 'vertical', label='Allocations')

# 4 Marks:
bc = bqplot.Bars(x = np.array(bc_data_grp.index), y = bc_data_grp.values, scales = {'x': x_scb, 'y': y_scb})

# Make the Bar Chart figure:
bc_fig = bqplot.Figure(marks = [bc], axes = [x_axb, y_axb], title = ('Allocations for selected cell'))


In [ ]:
xsedeDashboard = ipywidgets.VBox([selectedLabel, hm_fig, bc_fig])
xsedeDashboard

Looks like this is working great with the exception of a few things:
1. I need to figure out how to dynamically update the bar chart.
2. Selecting a heatmap cell that represents no allocations breaks the bar chart. The dynamic label shows 'nan' but the bar chart errors out.
3. general cleanup on the bar chart x-axis tick labels (overlapping)

I am out of time so I will be working on this after I submit this version. I will make sure the updates are in part 3 of the project.